In [ ]:
pip install python-telegram-bot==13.13

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: cachetools
    Found existing installation: cachetools 5.3.1
    Uninstalling cachetools-5.3.1:
      Successfully uninstalled cachetools-5.3.1


In [ ]:
pip install owlready2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 7.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for owlready2: filename=Owlready2-0.43-cp310-cp310-linux_x86_64.whl size=24044751 sha256=d4b42d959c149ef9f7fa4bdd32dedee8cdad72b95bd5391a5d3575a1a9df75ae
  Stored in directory: /root/.cache/pip/wheels/bc/9b/82/c07844dfbaa52dbf3352b18b0a27c348ee854df8b752990c53
Successfully built owlready2


In [ ]:
pip install RDFLib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.1 MB/s eta 0:00:00


In [ ]:
from owlready2 import *
import telegram
import time
import random
import logging
import pandas as pd
from owlready2.reasoning import sync_reasoner_pellet

from telegram import __version__ as TG_VER

try:
    from telegram import __version_info__
except ImportError:
    __version_info__ = (0, 0, 0, 0, 0)  # type: ignore[assignment]


from telegram import (
    InlineKeyboardButton,
    InlineKeyboardMarkup,
    Update,KeyboardButton,
    ReplyKeyboardMarkup,
    ReplyKeyboardRemove)

from telegram.ext import (
    Updater, CommandHandler,
    MessageHandler,
    Filters,CallbackQueryHandler,
    ConversationHandler,
    CallbackContext)

from owlready2 import get_ontology

In [ ]:
onto = get_ontology("file.owl").load()

with onto:
  # Rule 1: Menghitung BMI
  rule1 = Imp()
  rule1.set_as_rule('person(?p) , hasweight(?p, ?w) , hasheight(?p, ?h) , multiply(?wh, ?w, 10000) , multiply(?hm, ?h, ?h) , divide(?bmi, ?wh, ?hm) -> hasBMI(?p, ?bmi)')

  # Rule 2: Menghitung BMR untuk Laki-laki
  rule2 = Imp()
  rule2.set_as_rule('person(?p) , hasweight(?p, ?w) , hasheight(?p, ?h) , hasAge(?p, ?a) , hasGender(?p, "Laki-laki") , multiply(?k, ?w, 13.75) , multiply(?l, ?h, 5.003) , multiply(?m, ?a, 6.75) , add(?h1, ?k, 66.5) , add(?h2, ?h1, ?l) , subtract(?bmr, ?h2, ?m) -> hasBMR(?p, ?bmr)')

  # Rule 3: Menghitung BMR untuk Perempuan
  rule3 = Imp()
  rule3.set_as_rule('person(?p) , hasweight(?p, ?w) , hasheight(?p, ?h) , hasAge(?p, ?a) , hasGender(?p, "Wanita") , multiply(?k, ?w, 9.563) , multiply(?l, ?h, 1.850) , multiply(?m, ?a, 4.676) ,add(?h1, ?k, 655.1) , add(?h2, ?h1, ?l) , subtract(?bmr, ?h2, ?m) -> hasBMR(?p, ?bmr)')

  # Rule 4: person memiliki status normal (berdasarkan BMI)
  rule4 = Imp()
  rule4.set_as_rule('person(?p) , hasBMI(?p, ?bmi) , greaterThanOrEqual(?bmi, 18.5) , greaterThanOrEqual(22.9, ?bmi) -> normal(?p)')

  # Rule 5: person memiliki status underweight (berdasarkan BMI)
  rule5 = Imp()
  rule5.set_as_rule('person(?p) , hasBMI(?p, ?bmi) , greaterThan(18.5, ?bmi) -> underweight(?p)')

  # Rule 6: person memiliki status overweight (berdasarkan BMI)
  rule6 = Imp()
  rule6.set_as_rule('person(?p) , hasBMI(?p, ?bmi) , greaterThanOrEqual(?bmi, 23) , greaterThanOrEqual(24.9, ?bmi) -> overweight(?p)')

  # Rule 7: person memiliki status obesitas (berdasarkan BMI)
  rule7 = Imp()
  rule7.set_as_rule('person(?p) , hasBMI(?p, ?bmi) , greaterThanOrEqual(?bmi, 25.0) -> obesitas(?p)')

  # Rule 8: Menghitung kalori yang dibutuhkan berdasarkan BMR dan aktivitas fisik (Wanita - Little)
  rule8 = Imp()
  rule8.set_as_rule('person(?p) , hasGender(?p, "Wanita") , hasBMR(?p, ?bmr) , hasActivity(?p, "Little") , multiply(?c, ?bmr, 1.30) -> hasCalorieNeed(?p, ?c)')

  # Rule 9: Menghitung kalori yang dibutuhkan berdasarkan BMR dan aktivitas fisik (Wanita - Light)
  rule9 = Imp()
  rule9.set_as_rule('person(?p) , hasGender(?p, "Wanita") , hasBMR(?p, ?bmr) , hasActivity(?p, "Light") , multiply(?c, ?bmr, 1.55) -> hasCalorieNeed(?p, ?c)')

  # Rule 10: Menghitung kalori yang dibutuhkan berdasarkan BMR dan aktivitas fisik (Wanita - Moderate)
  rule10 = Imp()
  rule10.set_as_rule('person(?p) , hasGender(?p, "Wanita") , hasBMR(?p, ?bmr) , hasActivity(?p, "Moderate") , multiply(?c, ?bmr, 1.70) -> hasCalorieNeed(?p, ?c)')

  # Rule 11: Menghitung kalori yang dibutuhkan berdasarkan BMR dan aktivitas fisik (Wanita - Heavy)
  rule11 = Imp()
  rule11.set_as_rule('person(?p) , hasGender(?p, "Wanita") , hasBMR(?p, ?bmr) , hasActivity(?p, "Heavy") , multiply(?c, ?bmr, 2.00) -> hasCalorieNeed(?p, ?c)')

  # Rule 12: Menghitung kalori yang dibutuhkan berdasarkan BMR dan aktivitas fisik (Laki-laki - Little)
  rule12 = Imp()
  rule12.set_as_rule('person(?p) , hasGender(?p, "Laki-laki") , hasBMR(?p, ?bmr) , hasActivity(?p, "Little") , multiply(?c, ?bmr, 1.30) -> hasCalorieNeed(?p, ?c)')

  # Rule 13: Menghitung kalori yang dibutuhkan berdasarkan BMR dan aktivitas fisik (Laki-laki - Light)
  rule13 = Imp()
  rule13.set_as_rule('person(?p) , hasGender(?p, "Laki-laki") , hasBMR(?p, ?bmr) , hasActivity(?p, "Light") , multiply(?c, ?bmr, 1.56) -> hasCalorieNeed(?p, ?c)')

  # Rule 14: Menghitung kalori yang dibutuhkan berdasarkan BMR dan aktivitas fisik (Laki-laki - Moderate)
  rule14 = Imp()
  rule14.set_as_rule('person(?p) , hasGender(?p, "Laki-laki") , hasBMR(?p, ?bmr) , hasActivity(?p, "Moderate") , multiply(?c, ?bmr, 1.76) -> hasCalorieNeed(?p, ?c)')

  # Rule 15: Menghitung kalori yang dibutuhkan berdasarkan BMR dan aktivitas fisik (Laki-laki - Heavy)
  rule15 = Imp()
  rule15.set_as_rule('person(?p) , hasGender(?p, "Laki-laki") , hasBMR(?p, ?bmr) , hasActivity(?p, "Heavy") , multiply(?c, ?bmr, 2.10) -> hasCalorieNeed(?p, ?c)')

  # Rule 16: Menentukan kebutuhan kalori berdasarkan status dan kebutuhan kalori yang telah ditentukan (normal)
  rule16 = Imp()
  rule16.set_as_rule('person(?p) , normal(?p) , hasCalorieNeed(?p, ?a) -> hasCalorie(?p, ?a)')

  # Rule 17: Menentukan kebutuhan kalori berdasarkan status dan kebutuhan kalori yang telah ditentukan (Obesitas)
  rule17 = Imp()
  rule17.set_as_rule('person(?p) , obesitas(?p) , hasCalorieNeed(?p, ?a) , subtract(?s, ?a, 500) -> hasCalorie(?p, ?s)')

  # Rule 18: Menentukan kebutuhan kalori berdasarkan status dan kebutuhan kalori yang telah ditentukan (overweight)
  rule18 = Imp()
  rule18.set_as_rule('person(?p) , overweight(?p) , hasCalorieNeed(?p, ?a) , subtract(?s, ?a, 300) -> hasCalorie(?p, ?s)')

  # Rule 19: Menentukan kebutuhan kalori berdasarkan status dan kebutuhan kalori yang telah ditentukan (underweight)
  rule19 = Imp()
  rule19.set_as_rule('person(?p) , underweight(?p) , hasCalorieNeed(?p, ?a) , add(?s, ?a, 500) -> hasCalorie(?p, ?s)')

  # Rule 20: Membagi jumlah kalori harian seseorang menjadi porsi makanan yang berbeda (underweight)
  rule20 = Imp()
  rule20.set_as_rule(
      'multiply(?nbc, ?c, 0.25) , multiply(?nsc, ?c, 0.3) , multiply(?ndc, ?c, 0.2) , hasCalorie(?p, ?c) , underweight(?p) , multiply(?nlc, ?c, 0.25) -> LunchCal(?p, ?nlc) , SnackCal(?p, ?nsc) , BFCalorie(?p, ?nbc) , DinnerCal(?p, ?ndc)'
  )

  # Rule 21: Membagi jumlah kalori harian seseorang menjadi porsi makanan yang berbeda (normal)
  rule21 = Imp()
  rule21.set_as_rule(
      'multiply(?nbc, ?c, 0.25) , multiply(?nsc, ?c, 0.3) , multiply(?ndc, ?c, 0.2) , hasCalorie(?p, ?c) , normal(?p) , multiply(?nlc, ?c, 0.25) -> LunchCal(?p, ?nlc) , SnackCal(?p, ?nsc) , BFCalorie(?p, ?nbc) , DinnerCal(?p, ?ndc)'
  )

  # Rule 22: Membagi jumlah kalori harian seseorang menjadi porsi makanan yang berbeda (overweight)
  rule22 = Imp()
  rule22.set_as_rule(
      'multiply(?nbc, ?c, 0.35) , overweight(?p) , multiply(?nsc, ?c, 0.2) , multiply(?ndc, ?c, 0.2) , hasCalorie(?p, ?c) , multiply(?nlc, ?c, 0.25) -> LunchCal(?p, ?nlc) , SnackCal(?p, ?nsc) , BFCalorie(?p, ?nbc) , DinnerCal(?p, ?ndc)'
  )

  # Rule 23: Membagi jumlah kalori harian seseorang menjadi porsi makanan yang berbeda (obesitas)
  rule23 = Imp()
  rule23.set_as_rule(
      'multiply(?nbc, ?c, 0.35) , multiply(?nsc, ?c, 0.2) , obesitas(?p) , multiply(?ndc, ?c, 0.2) , hasCalorie(?p, ?c) , multiply(?nlc, ?c, 0.25) -> LunchCal(?p, ?nlc) , SnackCal(?p, ?nsc) , BFCalorie(?p, ?nbc) , DinnerCal(?p, ?ndc)'
  )

  # Rule 24: Menghitung kebutuhan air harian seseorang berdasarkan berat badan
  rule24 = Imp()
  rule24.set_as_rule(
      'person(?p) , hasweight(?p, ?w) , multiply(?wt, ?w, 30) , divide(?water, ?wt, 1000) -> hasWater(?p, ?water)'
  )

  #BREAKFAST MENU
  # Rule 25: untuk orang normal
  rule25 = Imp()
  rule25.set_as_rule('person(?p) , normal(?p),  Karbo(?f), hasNutrientCal(?f, ?nc) , add(?tc, ?nc, ?nc1)  , subtract(?bba, ?c, ?pcf1) , Vegetables(?f2) , roundHalfToEven(?pf1, ?bba) , greaterThan(?tcf, ?pf1) , greaterThan(?pf, ?tcf) , hasNutrientCal(?f1, ?nc1) , hasNutrientCal(?f2, ?nc2) , add(?tc1, ?tc, ?nc2) , BFCalorie(?p, ?c) , roundHalfToEven(?tcf, ?tc1) , multiply(?pcf1, ?c, 0.005) , Protein(?f1) , add(?ba, ?pcf1, ?c) , roundHalfToEven(?pf, ?ba) -> hasBreakfastMenu(?p, ?f) , hasBreakfastMenu(?f, ?f1) , hasBreakfastMenu(?f1, ?f2)'
  )

  # Rule 26: untuk orang underweight
  rule26 = Imp()
  rule26.set_as_rule('person(?p) , underweight(?p) , hasNutrientCal(?f, ?nc) , add(?tc, ?nc, ?nc1) , Karbo(?f) , subtract(?bba, ?c, ?pcf1) , Vegetables(?f2) , roundHalfToEven(?pf1, ?bba) , greaterThan(?tcf, ?pf1) , greaterThan(?pf, ?tcf) , hasNutrientCal(?f1, ?nc1) , hasNutrientCal(?f2, ?nc2) , add(?tc1, ?tc, ?nc2) , BFCalorie(?p, ?c) , roundHalfToEven(?tcf, ?tc1) , multiply(?pcf1, ?c, 0.005) , Protein(?f1) , add(?ba, ?pcf1, ?c) , roundHalfToEven(?pf, ?ba) -> hasBreakfastMenu(?p, ?f) , hasBreakfastMenu(?f, ?f1) , hasBreakfastMenu(?f1, ?f2)')

  # Rule 27: untuk orang overweight
  rule27 = Imp()
  rule27.set_as_rule('person(?p) , overweight(?p) , subtract(?bba, ?c, ?pcf1) , Vegetables(?f2) , greaterThan(?tcf, ?pf1) , hasNutrientCal(?f2, ?nc2) , add(?tc1, ?tc, ?nc2) , BFCalorie(?p, ?c) , multiply(?pcf1, ?c, 0.005) , Protein(?f1) , add(?ba, ?pcf1, ?c) , LevelNutrient(?f, LowFats) , roundHalfToEven(?pf, ?ba) , LevelNutrient(?f2, LowFats) , LevelNutrient(?f1, LowFats) , hasNutrientCal(?f, ?nc) , add(?tc, ?nc, ?nc1) , Karbo(?f) , roundHalfToEven(?pf1, ?bba) , greaterThan(?pf, ?tcf) , hasNutrientCal(?f1, ?nc1) , roundHalfToEven(?tcf, ?tc1) -> hasBreakfastMenu(?p, ?f) , hasBreakfastMenu(?f, ?f1) , hasBreakfastMenu(?f1, ?f2)')

  # Rule 28: untuk orang obesity
  rule28 = Imp()
  rule28.set_as_rule('person(?p) , obesitas(?p) , subtract(?bba, ?c, ?pcf1) , Vegetables(?f2) , greaterThan(?tcf, ?pf1) , hasNutrientCal(?f2, ?nc2) , add(?tc1, ?tc, ?nc2) , BFCalorie(?p, ?c) , multiply(?pcf1, ?c, 0.005) , Protein(?f1) , add(?ba, ?pcf1, ?c) , LevelNutrient(?f, LowFats) , roundHalfToEven(?pf, ?ba) , LevelNutrient(?f2, LowFats) , LevelNutrient(?f1, LowFats) , hasNutrientCal(?f, ?nc) , add(?tc, ?nc, ?nc1) , Karbo(?f) , roundHalfToEven(?pf1, ?bba) , greaterThan(?pf, ?tcf) , hasNutrientCal(?f1, ?nc1) , roundHalfToEven(?tcf, ?tc1) -> hasBreakfastMenu(?p, ?f) , hasBreakfastMenu(?f, ?f1) , hasBreakfastMenu(?f1, ?f2)')

  #LUNCH MENU
  # Rule 29: untuk orang normal
  rule29 = Imp()
  rule29.set_as_rule('person(?p) , normal(?p) , hasNutrientCal(?f, ?nc) , add(?tc, ?nc, ?nc1) , Karbo(?f) , subtract(?bba, ?c, ?pcf1) , Vegetables(?f2) , roundHalfToEven(?pf1, ?bba) , greaterThan(?tcf, ?pf1) , greaterThan(?pf, ?tcf) , hasNutrientCal(?f1, ?nc1) , hasNutrientCal(?f2, ?nc2) , add(?tc1, ?tc, ?nc2) , LunchCal(?p, ?c) , roundHalfToEven(?tcf, ?tc1) , multiply(?pcf1, ?c, 0.005) , Protein(?f1) , add(?ba, ?pcf1, ?c) , roundHalfToEven(?pf, ?ba) -> hasLunchMenu(?p, ?f) , hasLunchMenu(?f, ?f1) , hasLunchMenu(?f1, ?f2)'
  )

  # Rule 30: untuk orang underweight
  rule30 = Imp()
  rule30.set_as_rule('person(?p) , underweight(?p) , hasNutrientCal(?f, ?nc) , add(?tc, ?nc, ?nc1) , Karbo(?f) , subtract(?bba, ?c, ?pcf1) , Vegetables(?f2) , roundHalfToEven(?pf1, ?bba) , greaterThan(?tcf, ?pf1) , greaterThan(?pf, ?tcf) , hasNutrientCal(?f1, ?nc1) , hasNutrientCal(?f2, ?nc2) , add(?tc1, ?tc, ?nc2) , LunchCal(?p, ?c) , roundHalfToEven(?tcf, ?tc1) , multiply(?pcf1, ?c, 0.005) , Protein(?f1) , add(?ba, ?pcf1, ?c) , roundHalfToEven(?pf, ?ba) -> hasLunchMenu(?p, ?f) , hasLunchMenu(?f, ?f1) , hasLunchMenu(?f1, ?f2)')

  # Rule 31: untuk orang overweight
  rule31 = Imp()
  rule31.set_as_rule('person(?p) , overweight(?p) , subtract(?bba, ?c, ?pcf1) , Vegetables(?f2) , greaterThan(?tcf, ?pf1) , hasNutrientCal(?f2, ?nc2) , add(?tc1, ?tc, ?nc2) , multiply(?pcf1, ?c, 0.005) , Protein(?f1) , add(?ba, ?pcf1, ?c) , LevelNutrient(?f, LowFats) , roundHalfToEven(?pf, ?ba) , LevelNutrient(?f2, LowFats) , LevelNutrient(?f1, LowFats) , hasNutrientCal(?f, ?nc) , add(?tc, ?nc, ?nc1) , Karbo(?f) , roundHalfToEven(?pf1, ?bba) , greaterThan(?pf, ?tcf) , hasNutrientCal(?f1, ?nc1) , LunchCal(?p, ?c) , roundHalfToEven(?tcf, ?tc1) -> hasLunchMenu(?p, ?f) , hasLunchMenu(?f, ?f1) , hasLunchMenu(?f1, ?f2)')

  # Rule 32: untuk orang obesity
  rule32 = Imp()
  rule32.set_as_rule('person(?p) , obesitas(?p) , subtract(?bba, ?c, ?pcf1) , Vegetables(?f2) , greaterThan(?tcf, ?pf1) , hasNutrientCal(?f2, ?nc2) , add(?tc1, ?tc, ?nc2) , multiply(?pcf1, ?c, 0.005) , Protein(?f1) , add(?ba, ?pcf1, ?c) , LevelNutrient(?f, LowFats) , roundHalfToEven(?pf, ?ba) , LevelNutrient(?f2, LowFats) , LevelNutrient(?f1, LowFats) , hasNutrientCal(?f, ?nc) , add(?tc, ?nc, ?nc1) , Karbo(?f) , roundHalfToEven(?pf1, ?bba) , greaterThan(?pf, ?tcf) , hasNutrientCal(?f1, ?nc1) , LunchCal(?p, ?c) , roundHalfToEven(?tcf, ?tc1) -> hasLunchMenu(?p, ?f) , hasLunchMenu(?f, ?f1) , hasLunchMenu(?f1, ?f2)')

  #DINNER MENU
  # Rule 33: untuk orang normal
  rule33 = Imp()
  rule33.set_as_rule('person(?p) , normal(?p) , hasNutrientCal(?f, ?nc) , add(?tc, ?nc, ?nc1) , Karbo(?f) , subtract(?bba, ?c, ?pcf1) , Vegetables(?f2) , DinnerCal(?p, ?c) , roundHalfToEven(?pf1, ?bba) , greaterThan(?tcf, ?pf1) , greaterThan(?pf, ?tcf) , hasNutrientCal(?f1, ?nc1) , hasNutrientCal(?f2, ?nc2) , add(?tc1, ?tc, ?nc2) , roundHalfToEven(?tcf, ?tc1) , multiply(?pcf1, ?c, 0.005) , Protein(?f1) , add(?ba, ?pcf1, ?c) , roundHalfToEven(?pf, ?ba) -> hasDinnerMenu(?p, ?f) , hasDinnerMenu(?f, ?f1) , hasDinnerMenu(?f1, ?f2)'
  )

  # Rule 34: untuk orang underweight
  rule34 = Imp()
  rule34.set_as_rule('person(?p) , underweight(?p) , hasNutrientCal(?f, ?nc) , add(?tc, ?nc, ?nc1) , Karbo(?f) , subtract(?bba, ?c, ?pcf1) , Vegetables(?f2) , DinnerCal(?p, ?c) , roundHalfToEven(?pf1, ?bba) , greaterThan(?tcf, ?pf1) , greaterThan(?pf, ?tcf) , hasNutrientCal(?f1, ?nc1) , hasNutrientCal(?f2, ?nc2) , add(?tc1, ?tc, ?nc2) , roundHalfToEven(?tcf, ?tc1) , multiply(?pcf1, ?c, 0.005) , Protein(?f1) , add(?ba, ?pcf1, ?c) , roundHalfToEven(?pf, ?ba) -> hasDinnerMenu(?p, ?f) , hasDinnerMenu(?f, ?f1) , hasDinnerMenu(?f1, ?f2)')

  # Rule 35: untuk orang overweight
  rule35 = Imp()
  rule35.set_as_rule('person(?p) , overweight(?p) , subtract(?bba, ?c, ?pcf1) , Vegetables(?f2) , greaterThan(?tcf, ?pf1) , hasNutrientCal(?f2, ?nc2) , add(?tc1, ?tc, ?nc2) , multiply(?pcf1, ?c, 0.005) , Protein(?f1) , add(?ba, ?pcf1, ?c) , LevelNutrient(?f, LowFats) , roundHalfToEven(?pf, ?ba) , LevelNutrient(?f2, LowFats) , LevelNutrient(?f1, LowFats) , hasNutrientCal(?f, ?nc) , add(?tc, ?nc, ?nc1) , Karbo(?f) , DinnerCal(?p, ?c) , roundHalfToEven(?pf1, ?bba) , greaterThan(?pf, ?tcf) , hasNutrientCal(?f1, ?nc1) , roundHalfToEven(?tcf, ?tc1) -> hasDinnerMenu(?p, ?f) , hasDinnerMenu(?f, ?f1) , hasDinnerMenu(?f1, ?f2)')

  # Rule 36: untuk orang obesity
  rule36 = Imp()
  rule36.set_as_rule('person(?p) , obesitas(?p) , subtract(?bba, ?c, ?pcf1) , Vegetables(?f2) , greaterThan(?tcf, ?pf1) , hasNutrientCal(?f2, ?nc2) , add(?tc1, ?tc, ?nc2) , multiply(?pcf1, ?c, 0.005) , Protein(?f1) , add(?ba, ?pcf1, ?c) , LevelNutrient(?f, LowFats) , roundHalfToEven(?pf, ?ba) , LevelNutrient(?f2, LowFats) , LevelNutrient(?f1, LowFats) , hasNutrientCal(?f, ?nc) , add(?tc, ?nc, ?nc1) , Karbo(?f) , DinnerCal(?p, ?c) , roundHalfToEven(?pf1, ?bba) , greaterThan(?pf, ?tcf) , hasNutrientCal(?f1, ?nc1) , roundHalfToEven(?tcf, ?tc1) -> hasDinnerMenu(?p, ?f) , hasDinnerMenu(?f, ?f1) , hasDinnerMenu(?f1, ?f2)')

  #SNACK MENU
   # Rule 37: untuk orang normal (Snack A)
  rule37 = Imp()
  rule37.set_as_rule('person(?p) , normal(?p) , Cake(?f) , Fruit(?f1) , subtract(?bba, ?pcf, ?pcf1) , hasNutrientCal(?f, ?sc) , roundHalfToEven(?s, ?tc) , divide(?pcf, ?c, 3) , multiply(?pcf1, ?pcf, 0.005) , roundHalfToEven(?pf1, ?bba) , hasNutrientCal(?f1, ?sc1) , SnackCal(?p, ?c) , add(?tc, ?sc, ?sc1) , add(?ba, ?pcf1, ?pcf) , greaterThanOrEqual(?s, ?pf1) , roundHalfToEven(?pf, ?ba) , greaterThanOrEqual(?pf, ?s)  -> hasSnackA(?p, ?f) , hasSnackA(?f, ?f1)'
  )

  # Rule 38: untuk orang normal (Snack B)
  rule38 = Imp()
  rule38.set_as_rule('person(?p) , normal(?p) , Cake(?f) , Drink(?f1) , subtract(?bba, ?pcf, ?pcf1) , hasNutrientCal(?f, ?sc) , roundHalfToEven(?s, ?tc) , divide(?pcf, ?c, 3) , multiply(?pcf1, ?pcf, 0.005) , roundHalfToEven(?pf1, ?bba) , hasNutrientCal(?f1, ?sc1) , SnackCal(?p, ?c) , add(?tc, ?sc, ?sc1) , add(?ba, ?pcf1, ?pcf) , greaterThanOrEqual(?s, ?pf1) , roundHalfToEven(?pf, ?ba) , greaterThanOrEqual(?pf, ?s)  -> hasSnackB(?p, ?f) , hasSnackB(?f, ?f1)')

  # Rule 39: untuk orang underweight (Snack A)
  rule39 = Imp()
  rule39.set_as_rule('person(?p) , underweight(?p) , Snack(?f) , Fruit(?f1) , subtract(?bba, ?pcf, ?pcf1) , hasNutrientCal(?f, ?sc) , roundHalfToEven(?s, ?tc) , divide(?pcf, ?c, 3) , multiply(?pcf1, ?pcf, 0.005) , roundHalfToEven(?pf1, ?bba) , hasNutrientCal(?f1, ?sc1) , SnackCal(?p, ?c) , add(?tc, ?sc, ?sc1) , add(?ba, ?pcf1, ?pcf) , greaterThanOrEqual(?s, ?pf1) , roundHalfToEven(?pf, ?ba) , greaterThanOrEqual(?pf, ?s) -> hasSnackA(?p, ?f) , hasSnackA(?f, ?f1)')

  # Rule 40: untuk orang underweight (Snack B)
  rule40 = Imp()
  rule40.set_as_rule('person(?p) , underweight(?p) , Snack(?f) , Drink(?f1) , subtract(?bba, ?pcf, ?pcf1) , hasNutrientCal(?f, ?sc) , roundHalfToEven(?s, ?tc) , divide(?pcf, ?c, 3) , multiply(?pcf1, ?pcf, 0.005) , roundHalfToEven(?pf1, ?bba) , hasNutrientCal(?f1, ?sc1) , SnackCal(?p, ?c) , add(?tc, ?sc, ?sc1) , add(?ba, ?pcf1, ?pcf) , greaterThanOrEqual(?s, ?pf1) , roundHalfToEven(?pf, ?ba) , greaterThanOrEqual(?pf, ?s) -> hasSnackB(?p, ?f) , hasSnackB(?f, ?f1)')

   # Rule 41 : untuk orang overweight (Snack A & B)
  rule41 = Imp()
  rule41.set_as_rule('person(?p) , overweight(?p) , Fruit(?f) , Fruit(?f1) , LevelNutrient(?f, LowFats) , LevelNutrient(?f1, LowFats) , subtract(?bba, ?pcf, ?pcf1) , hasNutrientCal(?f, ?sc) , roundHalfToEven(?s, ?tc) , divide(?pcf, ?c, 2) , multiply(?pcf1, ?pcf, 0.005) , roundHalfToEven(?pf1, ?bba) , hasNutrientCal(?f1, ?sc1) , SnackCal(?p, ?c) , add(?tc, ?sc, ?sc1) , add(?ba, ?pcf1, ?pcf) , greaterThanOrEqual(?s, ?pf1) , roundHalfToEven(?pf, ?ba) , greaterThanOrEqual(?pf, ?s) -> hasSnackB(?p, ?f) , hasSnackB(?f, ?f1) , hasSnackA(?p, ?f) , hasSnackA(?f, ?f1)')

  # Rule 42: untuk orang obesity (Snack A & B)
  rule42 = Imp()
  rule42.set_as_rule('person(?p) , obesitas(?p) , Fruit(?f) , Fruit(?f1) , LevelNutrient(?f, LowFats) , LevelNutrient(?f1, LowFats) , subtract(?bba, ?pcf, ?pcf1) , hasNutrientCal(?f, ?sc) , roundHalfToEven(?s, ?tc) , divide(?pcf, ?c, 2) , multiply(?pcf1, ?pcf, 0.005) , roundHalfToEven(?pf1, ?bba) , hasNutrientCal(?f1, ?sc1) , SnackCal(?p, ?c) , add(?tc, ?sc, ?sc1) , add(?ba, ?pcf1, ?pcf) , greaterThanOrEqual(?s, ?pf1) , roundHalfToEven(?pf, ?ba) , greaterThanOrEqual(?pf, ?s) -> hasSnackB(?p, ?f) , hasSnackB(?f, ?f1) , hasSnackA(?p, ?f) , hasSnackA(?f, ?f1)')

  # Rule 43: Menentukan tingkat nutrisi Karbohidrat dari suatu menu makanan (HighCarbo)
  rule43 = Imp()
  rule43.set_as_rule(
      'Menu(?f) , hasNutrientCarbo(?f, ?n) , divide(?h, ?n, 300) , multiply(?DV, ?h, 100) , greaterThanOrEqual(?DV, 20) -> LevelNutrient(?f, HighCarbo)')

  # Rule 44: Menentukan tingkat nutrisi Karbohidrat dari suatu menu makanan (LowCarbo)
  rule44 =  Imp()
  rule44.set_as_rule(
      'Menu(?f) , hasNutrientCarbo(?f, ?n) , divide(?h, ?n, 300) , multiply(?DV, ?h, 100) , greaterThanOrEqual(5, ?DV) -> LevelNutrient(?f, LowCarbo)'
  )

  # Rule 45: : Menentukan tingkat nutrisi protein dari suatu menu makanan (HighProtein)
  rule45 = Imp()
  rule45.set_as_rule(
      'Menu(?f) , hasNutrientProtein(?f, ?n) , divide(?h, ?n, 50) , multiply(?DV, ?h, 100) , greaterThanOrEqual(?DV, 20) -> LevelNutrient(?f, HighProtein)'
  )

  # Rule 46: : Menentukan tingkat nutrisi protein dari suatu menu makanan (LowProtein)
  rule46 = Imp()
  rule46.set_as_rule(
      'Menu(?f) , hasNutrientProtein(?f, ?n) , divide(?h, ?n, 300) , multiply(?DV, ?h, 100) , greaterThanOrEqual(5, ?DV) -> LevelNutrient(?f, LowProtein)'
  )

  # Rule 47: Menentukan tingkat nutrisi lemak dari suatu menu makanan (HighFats)
  rule47 = Imp()
  rule47.set_as_rule(
      'Menu(?f) , hasNutrientFat(?f, ?n) , divide(?h, ?n, 78) , multiply(?DV, ?h, 100) , greaterThanOrEqual(?DV, 20) -> LevelNutrient(?f, HighFats)'
  )

  # Rule 48: Menentukan tingkat nutrisi lemak dari suatu menu makanan (LowFatss)
  rule48 = Imp()
  rule48.set_as_rule(
      'Menu(?f) , hasNutrientFat(?f, ?n) , divide(?h, ?n, 78) , multiply(?DV, ?h, 100) , greaterThanOrEqual(5, ?DV) -> LevelNutrient(?f, LowFats)'
  )

    # Rule 25: untuk orang normal
  rule25 = Imp()
  rule25.set_as_rule('person(?p) , normal(?p),  Karbo(?f1),hasNutrientCal(?f, ?nc) , add(?tc, ?nc, ?nc1)  , subtract(?bba, ?c, ?pcf1) , Vegetables(?f2) , roundHalfToEven(?pf1, ?bba) , greaterThan(?tcf, ?pf1) , greaterThan(?pf, ?tcf) , hasNutrientCal(?f1, ?nc1) , hasNutrientCal(?f2, ?nc2) , add(?tc1, ?tc, ?nc2) , BFCalorie(?p, ?c) , roundHalfToEven(?tcf, ?tc1) , multiply(?pcf1, ?c, 0.005) , Protein(?f1) , add(?ba, ?pcf1, ?c) , roundHalfToEven(?pf, ?ba) -> hasBreakfastMenu1(?p,?f) , hasBreakfastMenu1(?f, ?f1) , hasBreakfastMenu1(?f1, ?f2)'
  )


  onto.save("mood_ontology.owl")

In [ ]:
list(onto.classes())

[file.Menu,
 file.person,
 file.Protein,
 file.BMI_Level,
 file.normal,
 file.obesitas,
 file.overweight,
 file.underweight,
 file.Cake,
 file.Snack,
 file.Chips,
 file.Drink,
 file.FoodGroup,
 file.Fruit,
 file.Karbo,
 file.Type,
 file.Vegetables]

In [ ]:
# Create an instance of the Person class
import random
p1 = None

# Define the states for the conversation handler
WELCOME, PERSON_NAME, HAS_AGE, HAS_GENDER, HAS_HEIGHT, HAS_WEIGHT, HAS_ACTIVITY_LEVEL, PERSON_DATA_RETRIEVE = range(8)

# Define the command handlers for the Telegram bot
def start(update, context):
    context.bot.send_message(
        chat_id=update.effective_chat.id, text="🦋Selamat Datang di Chatbot Rekomendasi Makanan untuk Anak Sekolah (7-9 tahun)!")
    context.bot.send_message(
        chat_id=update.effective_chat.id, text="Kami akan mengajukan beberapa pertanyaan untuk memberikan rekomendasi yang tepat. Mohon diisi dengan benar🙏")
    return person_name(update, context)


def person_name(update, context):
    context.bot.send_message(
        chat_id=update.effective_chat.id, text="Silahkan masukkan nama:")
    return PERSON_NAME

def get_person_name(update, context):
    name = update.message.text
    context.user_data['name'] = name
    return has_age(update, context)


def has_age(update, context):
    context.bot.send_message(
        chat_id=update.effective_chat.id, text="Silahkan masukkan umur:")
    return HAS_AGE

def get_has_age(update, context):
    try:
        age = int(update.message.text)
        if age >= 7 and age <= 9:
            context.user_data['age'] = age
            return has_gender(update, context)
        else:
            context.bot.send_message(chat_id=update.effective_chat.id,
                                     text='Umur tidak valid. Silahkan masukkan umur antara 7 hingga 9 tahun.')
            return has_age(update, context)
    except ValueError:
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text='Silahkan masukkan umur dalam bentuk angka.')
        return has_age(update, context)


def has_gender(update, context):
    keyboard = [
        [InlineKeyboardButton("Laki-laki", callback_data='Laki-laki')],
        [InlineKeyboardButton("Wanita", callback_data='Wanita')],
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    context.bot.send_message(chat_id=update.effective_chat.id,
                             text="Silahkan masukkan jenis kelamin:", reply_markup=reply_markup)
    return HAS_GENDER

def get_has_gender(update, context):
    query = update.callback_query
    context.user_data['gender'] = query.data
    query.answer()
    return has_height(update, context)


def has_height(update, context):
    context.bot.send_message(
        chat_id=update.effective_chat.id, text="Silahkan masukkan tinggi badan (dalam cm):")
    return HAS_HEIGHT

def get_has_height(update, context):
    height = int(update.message.text)
    context.user_data['height'] = height
    return has_weight(update, context)


def has_weight(update, context):
    context.bot.send_message(
        chat_id=update.effective_chat.id, text="Silahkan masukkan berat badan (dalam kg):")
    return HAS_WEIGHT

def get_has_weight(update, context):
    weight = int(update.message.text)
    context.user_data['weight'] = weight
    return has_activity_level(update, context)


def has_activity_level(update, context):
    keyboard = [
        [InlineKeyboardButton('Little', callback_data="Little")],
        [InlineKeyboardButton('Light', callback_data="Light")],
        [InlineKeyboardButton('Moderate', callback_data="Moderate")],
        [InlineKeyboardButton('Heavy', callback_data="Heavy")],
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    context.bot.send_message(chat_id=update.effective_chat.id,
                             text="Pilih Level Aktivitas Fisik:\n\n"
                             "1.Little atau tidak ada olahraga\n"
                             "2.Olahraga Light atau olahraga 1-3 hari dalam seminggu\n"
                             "3.Olahraga Moderate atau olahraga 3-5 hari dalam seminggu\n"
                             "4.Olahraga Heavy atau olahraga 6-7 hari dalam seminggu\n",
                             reply_markup=reply_markup)
    return HAS_ACTIVITY_LEVEL

def get_has_activity_level(update, context):
    query = update.callback_query
    context.user_data['activity_level'] = query.data
    query.answer()
    return person_data_retrieve(update, context)


def person_data_retrieve(update, context):
    name = context.user_data['name']
    gender = context.user_data['gender']
    height = context.user_data['height']
    weight = context.user_data['weight']
    age = context.user_data['age']
    activity_level = context.user_data['activity_level']

    global p1
    p1 = onto.person(hasName=name, hasGender=gender, hasheight=height, hasweight=weight,
                            hasAge=age, hasActivity=activity_level)

    context.bot.send_message(chat_id=update.effective_chat.id,
                             text="Mohon tunggu sebentar sampai kami mengirimkan menu makan anda hari ini")
    run_reasoner(update, context)


def run_reasoner(update, context):
    name = context.user_data['name']
    # Run the reasoner
    sync_reasoner_pellet(infer_property_values=True, infer_data_property_values=True)

    # Print the property values of p1
    response = ""

    response += f"🌟 Nama: {name}\n"
    response += f"👶 Umur: {p1.hasAge}\n"
    response += f"🚻 Gender: {p1.hasGender}\n"
    response += f"💪 Body Mass Index (BMI): {p1.hasBMI:.2f}\n"
    response += f"🔥 Basal Metabolic Rate (BMR): {p1.hasBMR:.2f}\n"
    response += f"⚡️ Activity Level: {p1.hasActivity}\n\n"

    response += "🍳 Menu Rekomendasi:\n\n"

    # Breakfast
    karbo_menu = onto.KarboMenu.hasBreakfastMenu
    breakfast_item_karbo = random.choice(karbo_menu)
    response += f"🌄 Sarapan (KarboMenu): {str(breakfast_item_karbo).split('.')[-1]} (Size: {breakfast_item_karbo.hasServingSize[0]})\n"

    protein_menu = onto.ProteinMenu.hasBreakfastMenu
    breakfast_item_protein = random.choice(protein_menu)
    response += f"🌄 Sarapan (ProteinMenu): {str(breakfast_item_protein).split('.')[-1]} (Size: {breakfast_item_protein.hasServingSize[0]})\n"

    vegetables_menu = onto.VegetablesMenu.hasBreakfastMenu
    breakfast_item_vegetables = random.choice(vegetables_menu)
    response += f"🌄 Sarapan (VegetablesMenu): {str(breakfast_item_vegetables).split('.')[-1]} (Size: {breakfast_item_vegetables.hasServingSize[0]})\n"

    # Calculate total nutrient values for breakfast
    total_nutrient_carbo = breakfast_item_karbo.hasNutrientCarbo + \
        breakfast_item_protein.hasNutrientCarbo + \
        breakfast_item_vegetables.hasNutrientCarbo
    total_nutrient_cal = breakfast_item_karbo.hasNutrientCal + \
        breakfast_item_protein.hasNutrientCal + breakfast_item_vegetables.hasNutrientCal
    total_nutrient_fat = breakfast_item_karbo.hasNutrientFat + \
        breakfast_item_protein.hasNutrientFat + breakfast_item_vegetables.hasNutrientFat
    total_nutrient_protein = breakfast_item_karbo.hasNutrientProtein + \
        breakfast_item_protein.hasNutrientProtein + \
        breakfast_item_vegetables.hasNutrientProtein

    response += "🌄 Nutrisi Sarapan:\n"
    response += f"   - Karbohidrat: {total_nutrient_carbo}g\n"
    response += f"   - Kalori: {total_nutrient_cal}kcal\n"
    response += f"   - Lemak: {total_nutrient_fat}g\n"
    response += f"   - Protein: {total_nutrient_protein}g\n\n"

    # Lunch
    karbo_menu = onto.KarboMenu.hasLunchMenu
    lunch_item_karbo = random.choice(karbo_menu)
    response += f"🍱 Makan Siang (KarboMenu): {str(lunch_item_karbo).split('.')[-1]} (Size: {lunch_item_karbo.hasServingSize[0]})\n"

    protein_menu = onto.ProteinMenu.hasLunchMenu
    lunch_item_protein = random.choice(protein_menu)
    response += f"🍱 Makan Siang (ProteinMenu): {str(lunch_item_protein).split('.')[-1]} (Size: {lunch_item_protein.hasServingSize[0]})\n"

    vegetables_menu = onto.VegetablesMenu.hasLunchMenu
    lunch_item_vegetables = random.choice(vegetables_menu)
    response += f"🍱 Makan Siang (VegetablesMenu): {str(lunch_item_vegetables).split('.')[-1]} (Size: {lunch_item_vegetables.hasServingSize[0]})\n"

    # Calculate total nutrient values for lunch
    total_nutrient_carbo = lunch_item_karbo.hasNutrientCarbo + \
        lunch_item_protein.hasNutrientCarbo + lunch_item_vegetables.hasNutrientCarbo
    total_nutrient_cal = lunch_item_karbo.hasNutrientCal + \
        lunch_item_protein.hasNutrientCal + lunch_item_vegetables.hasNutrientCal
    total_nutrient_fat = lunch_item_karbo.hasNutrientFat + \
        lunch_item_protein.hasNutrientFat + lunch_item_vegetables.hasNutrientFat
    total_nutrient_protein = lunch_item_karbo.hasNutrientProtein + \
        lunch_item_protein.hasNutrientProtein + lunch_item_vegetables.hasNutrientProtein

    response += "🍱 Nutrisi Makan Siang:\n"
    response += f"   - Karbohidrat: {total_nutrient_carbo}g\n"
    response += f"   - Kalori: {total_nutrient_cal}kcal\n"
    response += f"   - Lemak: {total_nutrient_fat}g\n"
    response += f"   - Protein: {total_nutrient_protein}g\n\n"

    # Dinner
    karbo_menu = onto.KarboMenu.hasDinnerMenu
    dinner_item_karbo = random.choice(karbo_menu)
    response += f"🌆 Makan Malam (KarboMenu): {str(dinner_item_karbo).split('.')[-1]} (Size: {dinner_item_karbo.hasServingSize[0]})\n"

    protein_menu = onto.ProteinMenu.hasDinnerMenu
    dinner_item_protein = random.choice(protein_menu)
    response += f"🌆 Makan Malam (ProteinMenu): {str(dinner_item_protein).split('.')[-1]} (Size: {dinner_item_protein.hasServingSize[0]})\n"

    vegetables_menu = onto.VegetablesMenu.hasDinnerMenu
    dinner_item_vegetables = random.choice(vegetables_menu)
    response += f"🌆 Makan Malam (VegetablesMenu): {str(dinner_item_vegetables).split('.')[-1]} (Size: {dinner_item_vegetables.hasServingSize[0]})\n"

    # Calculate total nutrient values for dinner
    total_nutrient_carbo = dinner_item_karbo.hasNutrientCarbo + \
        dinner_item_protein.hasNutrientCarbo + dinner_item_vegetables.hasNutrientCarbo
    total_nutrient_cal = dinner_item_karbo.hasNutrientCal + \
        dinner_item_protein.hasNutrientCal + dinner_item_vegetables.hasNutrientCal
    total_nutrient_fat = dinner_item_karbo.hasNutrientFat + \
        dinner_item_protein.hasNutrientFat + dinner_item_vegetables.hasNutrientFat
    total_nutrient_protein = dinner_item_karbo.hasNutrientProtein + \
        dinner_item_protein.hasNutrientProtein + \
        dinner_item_vegetables.hasNutrientProtein

    response += "🌆 Nutrisi Makan Malam:\n"
    response += f"   - Karbohidrat: {total_nutrient_carbo}g\n"
    response += f"   - Kalori: {total_nutrient_cal}kcal\n"
    response += f"   - Lemak: {total_nutrient_fat}g\n"
    response += f"   - Protein: {total_nutrient_protein}g\n\n"

    # Snack A
    fruit_menu = onto.FruitMenu.hasSnackA
    chips_menu = onto.ChipsMenu.hasSnackA
    cake_menu = onto.CakeMenu.hasSnackA
    snack_a_item = random.choice(fruit_menu + chips_menu + cake_menu)
    response += f"🍪 Snack A: {str(snack_a_item).split('.')[-1]} (Size: {snack_a_item.hasServingSize[0]})\n"

    # Snack B
    fruit_menu = onto.FruitMenu.hasSnackB
    chips_menu = onto.ChipsMenu.hasSnackB
    cake_menu = onto.CakeMenu.hasSnackB
    snack_b_item = random.choice(fruit_menu + chips_menu + cake_menu)
    response += f"🍪 Snack B: {str(snack_b_item).split('.')[-1]} (Size: {snack_b_item.hasServingSize[0]})\n"

    # Snack C
    fruit_menu = onto.FruitMenu.hasSnackC
    chips_menu = onto.ChipsMenu.hasSnackC
    cake_menu = onto.CakeMenu.hasSnackC
    snack_c_item = random.choice(fruit_menu + chips_menu + cake_menu)
    response += f"🍪 Snack C: {str(snack_c_item).split('.')[-1]} (Size: {snack_c_item.hasServingSize[0]})\n"

    # Calculate total nutrient values for snacks
    total_nutrient_carbo = snack_a_item.hasNutrientCarbo + snack_b_item.hasNutrientCarbo + snack_c_item.hasNutrientCarbo
    total_nutrient_cal = snack_a_item.hasNutrientCal + snack_b_item.hasNutrientCal + snack_c_item.hasNutrientCal
    total_nutrient_fat = snack_a_item.hasNutrientFat + snack_b_item.hasNutrientFat + snack_c_item.hasNutrientFat
    total_nutrient_protein = snack_a_item.hasNutrientProtein + \
        snack_b_item.hasNutrientProtein + snack_c_item.hasNutrientProtein

    response += "🍪 Nutrisi Snack:\n"
    response += f"   - Karbohidrat: {total_nutrient_carbo}g\n"
    response += f"   - Kalori: {total_nutrient_cal}kcal\n"
    response += f"   - Lemak: {total_nutrient_fat}g\n"
    response += f"   - Protein: {total_nutrient_protein}g\n"

    context.bot.send_message(chat_id=update.effective_chat.id, text=response)



In [ ]:
# Set up the Telegram bot
TOKEN = "5912224124:AAF5jJ9LUJeUBeMa7ntjjb2ZUrd4Y1ftiag"
updater = Updater(token=TOKEN, use_context=True)
dispatcher = updater.dispatcher

logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)

logger = logging.getLogger(__name__)

def error(update, context):
    logger.warning('Update "%s" caused error "%s"', update, context.error)

# Register the command and message handlers
conv_handler = ConversationHandler(
    entry_points=[CommandHandler('start', start)],
    states={
        PERSON_NAME: [MessageHandler(Filters.text, get_person_name)],
        HAS_AGE: [MessageHandler(Filters.text, get_has_age)],
        HAS_GENDER: [CallbackQueryHandler(get_has_gender)],
        HAS_HEIGHT: [MessageHandler(Filters.text, get_has_height)],
        HAS_WEIGHT: [MessageHandler(Filters.text, get_has_weight)],
        HAS_ACTIVITY_LEVEL: [CallbackQueryHandler(get_has_activity_level)],
        PERSON_DATA_RETRIEVE: [MessageHandler(Filters.text, person_data_retrieve)]
    },
    fallbacks=[CommandHandler('start', start)]
)

dispatcher.add_handler(conv_handler)
dispatcher.add_error_handler(error)

# Start the bot
updater.start_polling()


/usr/local/lib/python3.10/dist-packages/telegram/ext/conversationhandler.py:288: UserWarning: If 'per_message=False', 'CallbackQueryHandler' will not be tracked for every message.
  warnings.warn(
